## Libraries:

In [3]:
from game import Game
import time
import os
import re
from collections import deque

## Load maps

In [4]:
def extract_map_files(directory):
    pattern = re.compile(r'^map(\d+)\.txt$')
    map_file_indices = []

    for file_name in os.listdir(directory):
        match = pattern.match(file_name)
        if match:
            map_file_indices.append(match.group(1))

    return [int(idx) for idx in map_file_indices]

def is_valid_input(map, indices, algorithm, solvers):
    valid_input = True
    if map not in indices:
        print(f"Map index out of range. Please choose within range {min(indices)} to {max(indices)}")
        valid_input = False
    if algorithm not in solvers.keys():    
        print(f"{algorithm} is not a defined algorithm. Please choose from", *[f"{solver} ({i+1})  " for i, solver in enumerate(solvers.keys())])
        valid_input = False
    return valid_input

def load_map(map_index):  
    file_name = "map" + str(map_index) + ".txt"
    with open('./assets/maps/' + file_name) as f:
        game_map = f.read()
    return game_map

map_file_indices = extract_map_files("./assets/maps/")

## BFS code:

In [5]:
from collections import deque
import time

def solver_bfs(game_map):
    game = Game(game_map)
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    queue = deque([(initial_state, "")])
    visited = set()
    visited.add(initial_state)
    directions = {'U': (-1, 0), 'D': (1, 0), 'L': (0, -1), 'R': (0, 1)}
    
    states_explored = 0
    start_time = time.perf_counter()
    
    while queue:
        (player_pos, box_positions), moves = queue.popleft()
        states_explored += 1
        
        game.set_player_position(player_pos)
        game.set_box_positions(box_positions)
        
        if game.is_game_won():
            return moves, states_explored, time.perf_counter() - start_time
        
        for move, (dy, dx) in directions.items():
            new_player_pos = game._Game__calculate_new_pos(player_pos, (dy, dx))  # Use portal-aware movement
            
            if game.is_wall(new_player_pos):  # Check if player can move
                continue
            
            new_box_positions = list(box_positions)
            if new_player_pos in box_positions:
                box_index = box_positions.index(new_player_pos)
                new_box_pos = game._Game__calculate_new_pos(new_player_pos, (dy, dx))  # Portal-aware box movement
                
                if not game.is_wall(new_box_pos) and new_box_pos not in box_positions:
                    new_box_positions[box_index] = new_box_pos
                else:
                    continue
            
            new_state = (new_player_pos, tuple(new_box_positions))
            if new_state not in visited:
                visited.add(new_state)
                queue.append((new_state, moves + move))
    
    return None, states_explored, time.perf_counter() - start_time


Description:
First, it initializes the game using the Game class and determines the initial position of the player and boxes. Then, it places this state, along with an empty move string, into a queue and adds it to a visited set to avoid revisiting states.
The algorithm explores states by dequeuing the oldest state from the queue. It sets the player and box positions in the game and checks if the game is won. If so, it returns the sequence of moves, the number of explored states, and the elapsed time.
For each possible movement direction (up, down, left, right), it calculates the new player position, considering portal mechanics. If the new position is a wall, it skips the move. If the player moves onto a box, it calculates the box’s new position. If the box's new position is valid (not a wall or another box), it updates the box's position.
If the new state (player position and box positions) is not visited, it adds it to the visited set and appends it to the queue with the updated move sequence.
If the queue becomes empty without finding a solution, it returns None and the number of explored states, indicating no BFS solution.

## BFS test:

In [ ]:
game_map = load_map(1)
game = Game(game_map)
game.display_map()

solution, visited_states, execution_time = solver_bfs(game_map)
if solution:
    print("BFS Solution found:", solution)
    game.apply_moves(solution)
    #game.display_map()
else:
    print("No BFS solution found.")

print("BFS Visited states:", visited_states)
print(f"BFS Execution time: {execution_time:.6f} seconds")

#Verification
#if game.is_game_won():
#    print("Game is won!")
#else:
#    print("Game is not won yet.")

#game.display_map()  


## DFS code:

In [7]:
from collections import deque
import time

def solver_dfs(game_map):
    game = Game(game_map)
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    stack = [(initial_state, "", 0)]  # (state, moves, depth)
    visited = {}
    directions = {'U': (-1, 0), 'D': (1, 0), 'L': (0, -1), 'R': (0, 1)}
    
    states_explored = 0
    start_time = time.perf_counter()
    best_solution = None
    best_solution_length = float('inf')
    
    while stack:
        (player_pos, box_positions), moves, depth = stack.pop()
        states_explored += 1
        
        game.set_player_position(player_pos)
        game.set_box_positions(box_positions)
        
        if game.is_game_won():
            if len(moves) < best_solution_length:
                best_solution = moves
                best_solution_length = len(moves)
            continue
        
        if depth >= best_solution_length:
            continue 
        
        for move, (dy, dx) in directions.items():
            new_player_pos = game._Game__calculate_new_pos(player_pos, (dy, dx))
            
            if game.is_wall(new_player_pos): 
                continue
            
            new_box_positions = list(box_positions)
            if new_player_pos in box_positions:
                box_index = box_positions.index(new_player_pos)
                new_box_pos = game._Game__calculate_new_pos(new_player_pos, (dy, dx))  
                
                if not game.is_wall(new_box_pos) and new_box_pos not in box_positions:
                    new_box_positions[box_index] = new_box_pos
                else:
                    continue
            
            new_state = (new_player_pos, tuple(new_box_positions))
            if new_state not in visited or len(moves) + 1 < visited[new_state]:
                visited[new_state] = len(moves) + 1
                stack.append((new_state, moves + move, depth + 1))
    
    return (best_solution if best_solution else None), states_explored, time.perf_counter() - start_time


Description: DFS explores possible moves by always diving deeper into a path before backtracking. This implementation uses a stack to track states, prioritizing last-in-first-out traversal. The algorithm starts with the initial player and box positions and iterates through possible movements. It checks if the goal state is reached, updating the best solution if a shorter path is found. To optimize, it prunes deeper searches if they exceed the best-known solution length. Movements are portal-aware, ensuring valid transitions. If a new state offers a shorter path than previously recorded, it is explored further. The function returns the shortest move sequence, the number of explored states, and the execution time. If no solution exists, it returns None.

## DFS test:

In [ ]:
game_map = load_map(1)
game = Game(game_map)
game.display_map()

solution, visited_states, execution_time = solver_dfs(game_map)
if solution:
    print("DFS Solution found:", solution)
    game.apply_moves(solution)
    #game.display_map()
else:
    print("No DFS solution found.")

print("DFS Visited states:", visited_states)
print(f"DFS Execution time: {execution_time:.6f} seconds")

#Verification
#if game.is_game_won():
#    print("Game is won!")
#else:
#    print("Game is not won yet.")

#game.display_map()  

## IDS code:

In [9]:
import time
from collections import deque

def dfs_with_depth_limit(game, depth_limit):
    queue = deque([(game.get_player_position(), tuple(game.get_box_locations()), "", 0)])  
    visited_states = set()

    while queue:
        player_pos, box_positions, path, depth = queue.popleft()

        if all(box == goal for box, goal in zip(box_positions, game.get_goal_locations())):
            return path, len(visited_states)

        if depth >= depth_limit:
            continue 

        state = (player_pos, box_positions)
        if state in visited_states:
            continue
        visited_states.add(state)

        for direction in ['U', 'L', 'D', 'R']:
            new_game = Game(game.get_map())
            new_game.set_player_position(player_pos)
            new_game.set_box_positions(box_positions)

            if new_game.apply_move(direction):
                new_state = (new_game.get_player_position(), tuple(new_game.get_box_locations()))
                if new_state not in visited_states:
                    queue.append((new_game.get_player_position(), tuple(new_game.get_box_locations()), path + direction, depth + 1))

    return None, len(visited_states)

def solver_ids(game_map):
    game = Game(game_map)
    depth_limit = 0
    start_time = time.time()

    while True:
        result, visited_states_count = dfs_with_depth_limit(game, depth_limit)

        if result:
            execution_time = time.time() - start_time
            return result, visited_states_count, execution_time  

        depth_limit += 1 


Description: The solver_ids function starts with a depth of 0 and repeatedly calls dfs_with_depth_limit, increasing the depth limit until a solution is found. In dfs_with_depth_limit, a depth-first search (DFS) explores possible moves (U, L, D, R), tracking visited states to prevent redundant computations.
Each move updates the game state, and if all boxes are on their goals, the function returns the move sequence. IDS ensures optimality by always finding the shortest path first while using less memory than Breadth-First Search (BFS). The output includes the shortest move sequence, the number of visited states, and the execution time.

## IDS test:

In [ ]:
game_map = load_map(1)
game = Game(game_map)
game.display_map()

solution, visited_states, execution_time = solver_ids(game_map)
if solution:
    print("IDS Solution found:", solution)
    game.apply_moves(solution)
    # game.display_map()
else:
    print("No IDS solution found.")

print("IDS Visited states:", visited_states)
print(f"IDS Execution time: {execution_time:.6f} seconds")

#Verification
#if game.is_game_won():
#    print("Game is won!")
#else:
#    print("Game is not won yet.")

#game.display_map()  


## A* code:

In [14]:
import heapq

def manhattan_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
def basic_heuristic(game):
    total_distance = sum(
        manhattan_distance(box, goal) for box, goal in zip(game.get_box_locations(), game.get_goal_locations())
    )
    player_to_box = min(
        (manhattan_distance(game.get_player_position(), box) for box in game.get_box_locations()),
        default=0
    )
    return total_distance + player_to_box
def improved_heuristic(game, weight=1):
    total_distance = 0
    for box, goal in zip(game.get_box_locations(), game.get_goal_locations()):
        total_distance += manhattan_distance(box, goal)

    player_to_nearest_box = min(
        (manhattan_distance(game.get_player_position(), box) for box in game.get_box_locations()),
        default=0
    )
    return total_distance + weight * player_to_nearest_box

def solver_astar(game_map, heuristic_func=improved_heuristic, weight=1):
    game = Game(game_map)
    priority_queue = []
    visited = set()
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    heapq.heappush(priority_queue, (0, "", initial_state))
    visited.add(initial_state)
    
    while priority_queue:
        cost, moves, (player_pos, box_positions) = heapq.heappop(priority_queue)
        
        game.set_player_position(player_pos)
        game.set_box_positions(box_positions)
        
        if game.is_game_won():
            return moves, len(visited)
        
        for move in "UDLR":
            new_game = Game(game_map)
            new_game.set_player_position(player_pos)
            new_game.set_box_positions(box_positions)
            
            if new_game.apply_move(move):
                new_state = (new_game.get_player_position(), tuple(new_game.get_box_locations()))
                if new_state not in visited:
                    visited.add(new_state) 
                    heuristic_value = heuristic_func(new_game, weight)
                    
                    new_cost = len(moves) + 1 + heuristic_value
                    heapq.heappush(priority_queue, (new_cost, moves + move, new_state))
    
    return "no solution", len(visited)


Description:
The Manhattan distance heuristic is a common choice for problems involving movement on a grid. In this problem, a simple heuristic could be:

$$
h = \sum_{i=1}^{n} \left( \text{Manhattan distance from player to box}_i + \text{Manhattan distance from box}_i \text{ to goal}_i \right)
$$

where:

The Manhattan distance is calculated as:
$$
|x_1 - x_2| + |y_1 - y_2|
$$

\( n \) is the number of boxes.

### Problems with the Basic Manhattan Distance Heuristic
- **Ignoring Obstacles**: This heuristic does not consider walls or other obstacles that might make the shortest path longer.
- **Ignoring Box Interactions**: If boxes block each other, the heuristic does not account for the extra moves needed to rearrange them.
- **Deadlocks**: The heuristic does not consider whether a box has been pushed into a position where it can no longer reach the goal.

### Improved Heuristic
A better heuristic should account for:
- Walls and obstacles.
- Box dependencies (i.e., preventing movements that lead to unsolvable positions).
- Identifying deadlocks early.

An improved version could be:

$$
h = \sum_{i=1}^{n} d(\text{box}_i, \text{goal}_i) + w \cdot d(\text{player}, \text{nearest box})
$$

where:

- d(box i , goal i) is the Manhattan distance from each box to its goal.
- d(player, nearest box) is the Manhattan distance from the player to the nearest box.
- w is a weight to prioritize moving the player efficiently.

**Why is the weight factor \( w \) necessary?**

If the value of this weight is very small (w approx 0), it means the heuristic focuses only on the movement of the boxes and ignores the player's position. In this case, A* may explore incorrect paths because it assumes the player can easily reach any box.

If the value of this weight is very large, it means the heuristic places too much importance on the player's position. This could cause the algorithm to focus more on the player's movement rather than solving the problem, which is not always helpful.

**How do we choose the appropriate value for \( w \)?**

✅ We select a balanced value, for example:

- \( w = 1 \) if the movement of the player and the movement of the boxes are of roughly equal importance.
- \( w > 1 \) if the player's movement is much more difficult, and we need to find better paths to get the player to the boxes.
- \( w < 1 \) if the primary focus is on the boxes and the player's movement is easy.

Additionally:
- Penalize states where a box is in a deadlock position (e.g., against a wall and not in a goal position).
- Use a modified distance metric (e.g., consider obstacles in a BFS-based distance calculation).

Now, let's implement the A* solver with the improved heuristic.

### A* Solver
Explanation of the A* Algorithm:
- **Priority Queue**: A min-heap is used to explore the most promising states first.
- **State Representation**: The state is defined by the player's position and the positions of all boxes.
- **Heuristic Calculation**: Uses the improved heuristic to estimate the cost to the goal.
- **Exploration**: Each possible move is attempted, and valid moves are pushed into the priority queue.
- **Termination**: The algorithm returns the optimal sequence of moves when the goal state is reached or reports "no solution" if no solution exists.

This improved heuristic considers both player and box positions while also accounting for movement complexity, making A* find an optimal solution efficiently. 


## A* Test:

In [ ]:
start_time = time.time()
game_map = load_map(2)
game = Game(game_map)
game.display_map()

solution, visited_states = solver_astar(game_map)
execution_time = time.time() - start_time

if solution != "no solution":
    print("A* Solution found:", solution)
    game.apply_moves(solution)
else:
    print("No A* solution found.")

print("A* Visited states:", visited_states)
print(f"A* Execution time: {execution_time:.6f} seconds")

#Verification
#if game.is_game_won():
#    print("Game is won!")
#else:
#    print("Game is not won yet.")

#game.display_map()  


## Weighted A* code:

In [13]:
import heapq

def manhattan_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
def improved_heuristic(game, weight=1):
    total_distance = 0
    for box, goal in zip(game.get_box_locations(), game.get_goal_locations()):
        total_distance += manhattan_distance(box, goal)

    player_to_nearest_box = min(
        (manhattan_distance(game.get_player_position(), box) for box in game.get_box_locations()),
        default=0
    )
    return total_distance + weight * player_to_nearest_box


def solver_weighted_astar(game_map, heuristic_func=improved_heuristic, weight=1):
    game = Game(game_map)
    priority_queue = []
    visited = set()
    initial_state = (game.get_player_position(), tuple(game.get_box_locations()))
    heapq.heappush(priority_queue, (0, "", initial_state))
    visited.add(initial_state)
    
    while priority_queue:
        cost, moves, (player_pos, box_positions) = heapq.heappop(priority_queue)
        
        game.set_player_position(player_pos)
        game.set_box_positions(box_positions)
        
        if game.is_game_won():
            return moves, len(visited)
        
        for move in "UDLR":
            new_game = Game(game_map)
            new_game.set_player_position(player_pos)
            new_game.set_box_positions(box_positions)
            
            if new_game.apply_move(move):
                new_state = (new_game.get_player_position(), tuple(new_game.get_box_locations()))
                if new_state not in visited:
                    visited.add(new_state) 
                    heuristic_value = heuristic_func(new_game, weight)
                    new_cost = len(moves) + 1 + weight * heuristic_value
                    heapq.heappush(priority_queue, (new_cost, moves + move, new_state))
    
    return "no solution", len(visited)

**Description:**
Weighted A* is a variant of the A* algorithm that prioritizes heuristic information more aggressively by multiplying the heuristic function by a weight factor 𝑤. This often speeds up the search but can lead to suboptimal solutions if w is too large.
The cost function in Weighted A* is defined as:
$$
f(n)=g(n)+w×h(n)
$$

f(n) = Total estimated cost from the start to the goal through node 
g(n) = Cost from the start node to 
h(n) = Heuristic estimate of the cost from 
w = Weight factor that amplifies the heuristic’s influence

**Functions and Their Roles in the Code:**

1. Heuristic Function: h(n)
The heuristic estimates the remaining cost to reach the goal.
Basic Manhattan Distance Heuristic: ( Common choices)
$$
ℎ
(
𝑛
)
=
∑
𝑖
Manhattan distance between box 
𝑖
 and its goal
$$
Improved Heuristic:
Adds distance from the player to the nearest box to prioritize reachable boxes.
Penalizes boxes that are stuck in corners or unreachable states.
Uses weighted multipliers to balance between efficiency and accuracy.


2. Cost Function: g(n)
it is the actual cost from the start node to the current node. In this case, each move costs 1, so:
$$
g(n)=number of moves taken so far
$$
This ensures that shorter paths are preferred when expanding nodes.



**Tuning Weighted A for Optimal Performance:**
1. Reduce the Weight: 
if w is too high, the search will prioritize the heuristic too much and ignore shorter paths.
Solution: w must be 1.1 – 1.5 for better balance

2. Improve the Heuristic Function:
Identify trapped boxes: If a box is in a corner and cannot reach the goal, assign it a very high heuristic value.
Consider box sequences: If a box is blocking another box’s path, add a penalty


3. Increase g(n) Consideration:
If the algorithm is prioritizing the heuristic too much, it might ignore paths that have a lower real cost.
Solution: Make sure g(n) properly affects the cost:
f(n) = g(n) + w*h(n)
If needed, increase the effect of g(n) by tweaking the weight of w.


**Conclusion:**

1. Weighted A* is a faster but sometimes suboptimal version of A*.
2. Lowering w helps find the shortest path.
3. Better heuristics (considering box traps, player movement) improve accuracy.
4. Balancig g(n) and h(n) ensures optimal solutions.


## Weighted A* test:

In [ ]:
import time

start_time = time.time()
game_map = load_map(5) 
game = Game(game_map)
game.display_map()
game.is_game_won();
solution, visited_states = solver_weighted_astar(game_map,weight=1.5)
execution_time = time.time() - start_time

if solution != "no solution":
    print("Weighted A* Solution found:", solution)
    game.apply_moves(solution)
else:
    print("No Weighted A* solution found.")

print("Weighted A* Visited states:", visited_states)
print(f"Weighted A* Execution time: {execution_time:.6f} seconds")

#Verification
#if game.is_game_won():
#    print("Game is won!")
#else:
#    print("Game is not won yet.")

#game.display_map()  

